In [1]:
# Need to restart runtime before this step
import time
import torch
import numpy as np
import scipy.ndimage
import scipy.io
import matplotlib.pyplot as plt
import deepwave
import os
import csv
from sklearn.feature_extraction import image
from skimage.transform import resize
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
# General info
# using Adam Optimization
# Shuffling sources every epoch
# SCHUSTER's MARMOUSI MODEL

# General parameters
GPU=2     # which GPU to use
plot=False # Plot Intermediate results
sigma=5.0   # Initial model smoothness  

# Inversion parameters
num_batches = 2
num_epochs = 5000

#gamma = 300  # Does not apply, Adam uses lr (learning rate) instead of gamma
# Adam learning rate
lr = 4

# Modeling parameters
freq = 8
dx = 10
dt = 0.001
nt = 4001
num_shots = 18
num_receivers_per_shot = 200

# receiver_spacing = # computed below from ny,dx and num_receivers_per_shot

# Close the ouput.txt file with saved parameters
%logstop 

Logging hadn't been started.


In [4]:
plot=True
# Load the true model
filename='../FWI_Ricardo/Marm151_401.mat'
mat=scipy.io.loadmat(filename)['vel']
mat = resize(mat,(151,200))
#mat = np.loadtxt('marmhard2.dat')
#mat = np.pad(mat,((10,0),(0,0)),'edge') #RB add some water layer
#RB Get model size
ny = np.size(mat,1);
nz = np.size(mat,0);
#RB compute true model mean value and width for future model normalization
max1=np.max(mat)
min1=np.min(mat)
med=(max1+min1)/2.
wid=(max1-min1)/2.

print('vmin=',min1,'vmax=',max1,'vmed=',med,'vwidth=',wid)


vmin= 1500.0 vmax= 3550.000244140625 vmed= 2525.0001220703125 vwidth= 1025.0001220703125


In [5]:
# Specify which GPU to use (in multi GPU machines)
torch.cuda.set_device(GPU)  #RB Necessary if device <> 0
GPU_string='cuda:'+str(GPU)
device = torch.device(GPU_string)

In [6]:
# Set parameters
num_dims = 2
#num_shots = 32
num_sources_per_shot = 1
#num_receivers_per_shot = 200

#RB source_spacing = 25.0
#RB receiver_spacing = 10.0
source_spacing = ny * dx / num_shots
receiver_spacing = ny * dx / num_receivers_per_shot
#receiver_spacing = 8.

print('ny=',ny,'nz=',nz,'nt=',nt,'dx=',dx,'freq=',freq)
print('shots=',num_shots,'recs/shot=',num_receivers_per_shot, \
      'ds=',source_spacing,'dr=',receiver_spacing)


ny= 200 nz= 151 nt= 4001 dx= 10 freq= 8
shots= 18 recs/shot= 200 ds= 111.11111111111111 dr= 10.0


In [26]:
# Create arrays containing the source and receiver locations
# x_s: Source locations [num_shots, num_sources_per_shot, num_dimensions]
# x_r: Receiver locations [num_shots, num_receivers_per_shot, num_dimensions]
x_s = torch.zeros(num_shots, num_sources_per_shot, num_dims)
x_s[:, 0, 0] = torch.linspace(0,(ny-1)*dx,num_shots)
x_s[:, 0, 1] = 10. #RB Set source depth

po = len(torch.arange(4,num_receivers_per_shot))
x_r = torch.zeros(num_shots, po, num_dims)
x_r[:, :, 0] = torch.arange(4,num_receivers_per_shot).float() * receiver_spacing
x_r[:, :, 1] = 15.

source_amplitudes_true = (deepwave.wavelets.ricker(freq, nt, dt, 1/freq)
    .repeat(num_shots, 1, 1)
    .to(device))

In [27]:
receiver_spacing

10.0

In [28]:
#RB Convert models to PyTorch Tensors
model_true = torch.Tensor(mat) # Convert to a PyTorch Tensor

In [29]:
out = deepwave.scalar(model_true.to(device), dx, dt, source_amplitudes=source_amplitudes_true,
             source_locations=x_s.to(device),
             receiver_locations=x_r.to(device),
             accuracy=8,
             pml_freq=freq)

RuntimeError: Locations must be within model.